# Variable

In [ ]:
yolo_image_size = 640
image_file_folder = <Folder-path>
image_file_name = "test"
image_file_type = "png"
endpoint_url = <endpoint_url>
font_path = "./font/Yagora.ttf"

# Code

In [2]:
import json
import numpy as np
import cv2
import requests
from PIL import Image, ImageDraw, ImageFont

In [3]:
CLASSES = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat", "traffic light", 
    "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", "dog", "horse", "sheep", "cow", 
    "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee", 
    "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", 
    "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple", 
    "sandwich", "orange", "broccoli", "car|rot", "hot dog", "pizza", "donut", "cake", "chair", "couch", 
    "potted plant", "bed", "dining table", "toilet", "tv", "laptop", "mouse", "remote", "keyboard", 
    "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors", 
    "teddy bear", "hair drier", "toothbrush"]

In [4]:
original_image = cv2.imread("{}{}.{}".format(image_file_folder, image_file_name, image_file_type))

json_data = {}
json_data['data'] = {}
json_data['data']["ndarray"] = [original_image.tolist()]
with open('{}{}.json'.format(image_file_folder, image_file_name), 'w') as f:
    json.dump(json_data, f)
print("Done for transfer image to json file.")
print("json file path: {}{}.json".format(image_file_folder, image_file_name))

Done for transfer image to json file.
json file path: /home/jovyan/yolov7_test/data/test.json


In [5]:
def retry_requests(endpoint_url, json_data, retry_times = 20):
    for time in range(1, retry_times):
        response = requests.post(endpoint_url, json=json_data)
        if response.status_code == 200:
            print("Got status code 200. Return response.")
            return response
        elif response.status_code == 413:
            print("Got status code 413. Need to setting nginx ingress controller.")
            return response
        elif response.status_code == 504:
            print("Got status code 504. Retry the request. Time: {}/{}.".format(time, retry_times))
        else:
            return response

In [6]:
response = retry_requests(endpoint_url, json_data)
if response.status_code == 200:
    response_json = json.loads(response.text)
    pred = response_json['data']['ndarray']
else:
    print(response.text)

Got status code 200. Return response.


In [7]:
image = Image.open("{}{}.{}".format(image_file_folder, image_file_name, image_file_type))
font = ImageFont.truetype(font_path, 30)
draw = ImageDraw.Draw(image)
for x1, y1, x2, y2, conf, class_id in pred:
    draw.rectangle(((x1, y1), (x2, y2)), width=2)
    draw.text((x1, y1), "{}, {}".format(CLASSES[int(class_id)], round(conf, 3)), font = font)

image.save("{}{}_prediction.{}".format(image_file_folder, image_file_name, image_file_type))

# Result

In [8]:
print("You can check the result image: {}".\
      format("{}{}_prediction.{}".format(image_file_folder, image_file_name, image_file_type)))

You can check the result image: /home/jovyan/yolov7_test/data/test_prediction.png
